## Modules

In [1]:
import os
import sys
sys.path.append('..')
import shutil
from glob import glob
import re
from time import sleep

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from itertools import product

from hkexoi import *
from hkexoi.hktradedates import *
from hkexoi.database import *

import requests
from bs4 import BeautifulSoup

oldpath = r'C:\Users\willbillion\Downloads'

volpath = os.path.join(filepath, 'ivhv')
sourcepath = os.path.join(volpath, 'source')
if not os.path.exists(sourcepath):
    os.makedirs(sourcepath)
    
from selenium import webdriver
browser = webdriver.Chrome(executable_path=r'D:\chromedriver.exe')

<ipython-input-1-e4130e40ee70>:29: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=r'D:\chromedriver.exe')


## HKEX HV vs IV for a single stock symbol

### Download flow

In [2]:
linkprefix = 'https://www.hkex.com.hk/eng/sorc/options/statistics_hv_iv.aspx'

ltddate = datetime.today() - timedelta(days=1, hours=6)
ltdstr = ltddate.strftime('%y%m%d')

r = requests.get(linkprefix)
content = r.content
soup = BeautifulSoup(content, "lxml")
souptable = soup.find_all('table', {'class': 'noncms_table'})
tabletext = souptable[0].text
numregex = re.compile(r'\d{5}')
numlist = numregex.findall(tabletext)

for symbol in numlist:
    newfile = os.path.join(sourcepath, f'ivhv_{symbol}-{ltdstr}.csv')
    if not os.path.exists(newfile):
        linksuffix = f'?ucode={symbol}'
        link = linkprefix + linksuffix
        browser.get(link)
        elem = browser.find_element_by_link_text('Export to CSV')
        elem.click()
        sleep(0.2)
        oldname = os.path.join(oldpath, 'statistics_hv_iv.csv')
        newname = os.path.join(sourcepath, 'statistics_hv_iv.csv')
        newfile = os.path.join(sourcepath, f'ivhv_{symbol}-{ltdstr}.csv')
        shutil.move(oldname, sourcepath)
        os.rename(newname, newfile)
        print(f'{symbol} ready.')
    else:
        print(f'{symbol} already there.')

00001 already there.
00002 already there.
00003 already there.
00004 already there.
00005 already there.
00006 already there.
00011 already there.
00012 already there.
00016 already there.
00017 already there.
00019 already there.
00023 already there.
00027 already there.
00066 already there.
00135 already there.
00151 already there.
00175 already there.
00241 already there.
00267 already there.
00268 already there.
00285 already there.
00288 already there.
00293 already there.
00358 already there.
00386 already there.
00388 already there.
00390 already there.
00489 already there.
00669 already there.
00688 already there.
00700 already there.
00728 already there.
00753 already there.
00762 already there.
00788 already there.
00823 already there.
00857 already there.
00868 already there.
00883 already there.
00902 already there.
00914 already there.
00939 already there.
00941 already there.
00968 already there.
00981 already there.
00992 already there.
00998 already there.
01024 already

### Parsing the downloaded CSV and integrate into existing master CSV

In [3]:
filelist = glob(os.path.join(sourcepath, 'ivhv_*.csv'))

renamedict = {'Trading date': 'date', ' IV (%)': 'iv'}
renamedict.update({f' HV{num} (%)': f'hv{num}' for num in [10, 30, 60, 90]})

vollist = []

for file in filelist:
    csvcontent = open(file, encoding="utf8")
    oldrowlist = csvcontent.readlines()[2:]
    newrowlist = []
    for row in oldrowlist:
        row = tuple(row.replace('\n', '').split(','))
        newrowlist.append(row)

    df = pd.DataFrame.from_records(newrowlist[1:], columns=newrowlist[0])
    df.rename(columns=renamedict, inplace=True)
    df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
    df.set_index('date', inplace=True)
    df.sort_index(inplace=True)
    for col in df.columns:
        df[col] = pd.to_numeric(df[col])
        df[col] = np.round(df[col], 4)
    symbol = file[-16:-11]
    volfile = os.path.join(volpath, f'ivhv_{symbol}.csv')
    
    if not os.path.exists(volfile):
        df.to_csv(volfile)
    else:
        dfcsv = pd.read_csv(volfile, header=0)
        dfcsv['date'] = pd.to_datetime(dfcsv['date'], format='%Y-%m-%d')
        dfcsv.set_index('date', inplace=True)
        dfcsv.sort_index(inplace=True)
        for col in dfcsv.columns:
            dfcsv[col] = pd.to_numeric(dfcsv[col])
            dfcsv[col] = np.round(dfcsv[col], 4)
        newindex = df.index.difference(dfcsv.index)
        dfnew = pd.concat([dfcsv, df.loc[newindex]], axis=0)
        dfnew.sort_index(inplace=True)
        dfnew.to_csv(volfile)